In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets
import torch.nn.functional as F

In [3]:
#### Hyperparameter
batch_size = 16
patch_size = 16

In [ ]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
])


train_ds = datasets.CIFAR100(root="./data", download=True, train=True, transform=transform)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

In [7]:
print("data batch size: {}".format(next(iter(train_dl))[0].size()))

data batch size: torch.Size([16, 3, 224, 224])


In [ ]:
## step 1: patchify Layer
class PatchLayer(nn.Module):
    def __init__(self, patch_size):
        self.patch_size = patch_size

    # input = (N, C, H, W) ===> return = (N, CHW/ P^2, P^2)
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        pass

In [ ]:
## step 2: Map Layer
class LinearMap(nn.Module):
    def __init__(self, hidden_size):
        pass

    # input = (N, S, d) ===> return = (N, S, d')
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        pass

In [ ]:
## step 3: Positional Encoding
class PositionalEncoding(nn.Module):
    def __init__(self):
        pass

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        pass

In [ ]:
## step 4: Transformer Block

In [ ]:
## step 5: classifier Block

In [ ]:
class VIT(nn.Module):
    pass